In [1]:
from estnltk.wordnet import wn
import sqlite3
from lxml import etree

In [2]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [3]:
sset_db = 'wordnet/data/xml_synsets.db'
relation_db = 'wordnet/data/xml_relations.db'
xml_entries = 'wordnet/data/xml_entries.db'

In [4]:
end_vrtx = []
start_vrtx = []
start_sset = []
end_sset = []
rel_type = []

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def sqlTables(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS rel_table(

                                        start_estwn TEXT NOT NULL,
                                        end_estwn TEXT NOT NULL,
                                        relation TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")

In [ ]:
def synset_str(synset_word, cursor):
    cursor.execute('SELECT pos, sense FROM xml_synsets WHERE sourceSense = ?',(synset_word,))
    data = cursor.fetchone()
    if data is not None:
        var = "." + data[0] + "." + str(data[1])
        synset_word += var
    else: return None
    
    return synset_word
    

In [6]:
def fetch_sset_word(estwnId, entryCursor):
    '''
    
    '''
    print("estwnId in fetch word: ", estwnId)
    ssetList = []
    entryCursor.execute("SELECT DISTINCT writtenForm FROM xml_lexicalEntry WHERE estwn_id = ?",(estwnId,))
    writtenForm = entryCursor.fetchone()
    print(writtenForm)

    return writtenForm[0]

In [7]:
def fetch_id(sset_word,cursor):
    
    print("sset_word: ", sset_word)
    cursor.execute('SELECT id FROM xml_synsets WHERE writtenForm = ?',(sset_word,))
    ssetId = cursor.fetchone()
    print("return id:",ssetId[0])
    
    return ssetId[0]

In [2]:
def get_synsets(start_estwn, end_estwn, end_sset, rel_type):

    synsetList  = tree.xpath('/LexicalResource/Lexicon/Synset')
    i=0
    for sset in synsetList: 
        end = sset.attrib['id']        
        if len(sset) > 1:
            j=0
            for subset in sset:
                try:
                    start_estwn.append(subset.attrib['target'])
                    rel_type.append(subset.attrib['relType'])
                    end_estwn.append(end)
                except:
                    #print("except @ {}.{}" .format(i,j))
                    if j > 0:
                        try:
                        
                            end_estwn.append(end)
                            start_estwn.append('null')
                            rel_type.append('null')
                            #print("pass {}.{}".format(i,j))
                        except: pass  
                j+=1
                
        if len(sset) == 1:
            end_estwn.append(end)
            end_sset.append(ssetWord)
            try:
                start_estwn.append( sset[0].attrib['target'] )
                rel_type.append( sset[0].attrib['relType'] )            
            except:
                #print("else pass @ ", i)
                start_estwn.append("null")
                rel_type.append("null")
                pass
        i+=1

In [69]:
def upload_relations(db_file):
    
    sqlTables(db_file)
    conn = create_connection(db_file)
    cursor = conn.cursor()
    
    with conn:
        for i in range(len(end_estwn)):
            
            if i >= len(start_estwn):
                    start = 'null'
                    rel = 'null'
            else:
                start = start_estwn[i]
                rel = rel_type[i]
            cursor.execute("INSERT INTO rel_table(start_estwn, end_estwn, sourceSense, relation) VALUES(?,?,?,?)"\
                                                            ,(start, end_estwn[i], end_sset[i], rel))
            conn.commit()

In [ ]:
start_estwn = []
end_sset    = []
end_estwn   = []
rel_type    = []
get_synsets(start_estwn, end_estwn, end_sset, rel_type)

In [95]:
upload_relations(relation_db)